In [2]:
from src.experiments import ant_experiment_sac, halfchetah_experiment_sac, ant_experiment_td3

ant_experiment_td3({'mujoco': 0.5, 'brax': 0.5}, n_runs=2)

No GPU found, using CPU
Registering BraxHalfCheetah-v0...
✅ Successfully registered BraxHalfCheetah-v0
Registering BraxAnt-v0...
✅ Successfully registered BraxAnt-v0
Registering BraxHumanoid-v0...
✅ Successfully registered BraxHumanoid-v0
Registering BraxWalker2d-v0...
✅ Successfully registered BraxWalker2d-v0
Registering BraxHopper-v0...
✅ Successfully registered BraxHopper-v0
Registering BraxReacher-v0...
✅ Successfully registered BraxReacher-v0
✅ Brax environments registration complete
✅ Created mujoco environment: Ant-v4
Initializing fast Brax environment: ant


/Users/felixpasewaldt/opt/anaconda3/envs/hysarl/lib/python3.10/site-packages/gymnasium/envs/registration.py:519: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


✅ Fast Brax environment initialized with JIT compilation
  Observation space: Box(-inf, inf, (27,), float32)
  Action space: Box(-1.0, 1.0, (8,), float32)
✅ Created brax environment: BraxAnt-v0
✅ Created 2 environment types for sequential training.

Episode 1/500 Summary:
mujoco:  Mean Reward: 963.723  Mean Steps: 1000.0  Avg Duration: 0.204s
brax:  Mean Reward: 80.893  Mean Steps: 76.0  Avg Duration: 5.554s

Episode 11/500 Summary:
mujoco:  Mean Reward: 736.020  Mean Steps: 1000.0  Avg Duration: 0.218s
brax:  Mean Reward: 12.239  Mean Steps: 44.0  Avg Duration: 0.061s

Episode 21/500 Summary:
mujoco:  Mean Reward: 563.177  Mean Steps: 1000.0  Avg Duration: 0.174s
brax:  Mean Reward: 20.526  Mean Steps: 29.0  Avg Duration: 0.050s

Episode 31/500 Summary:
mujoco:  Mean Reward: 506.553  Mean Steps: 1000.0  Avg Duration: 0.197s
brax:  Mean Reward: -5.943  Mean Steps: 66.0  Avg Duration: 0.037s

Episode 41/500 Summary:
mujoco:  Mean Reward: 195.748  Mean Steps: 169.0  Avg Duration: 0.176s


/Users/felixpasewaldt/opt/anaconda3/envs/hysarl/lib/python3.10/site-packages/gymnasium/envs/registration.py:519: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


✅ Fast Brax environment initialized with JIT compilation
  Observation space: Box(-inf, inf, (27,), float32)
  Action space: Box(-1.0, 1.0, (8,), float32)
✅ Created brax environment: BraxAnt-v0
✅ Created 2 environment types for sequential training.

Episode 1/500 Summary:
mujoco:  Mean Reward: 949.135  Mean Steps: 1000.0  Avg Duration: 0.194s
brax:  Mean Reward: 37.307  Mean Steps: 89.0  Avg Duration: 4.683s

Episode 11/500 Summary:
mujoco:  Mean Reward: 888.984  Mean Steps: 1000.0  Avg Duration: 0.173s
brax:  Mean Reward: 22.873  Mean Steps: 23.0  Avg Duration: 0.048s

Episode 21/500 Summary:
mujoco:  Mean Reward: 604.186  Mean Steps: 1000.0  Avg Duration: 0.188s
brax:  Mean Reward: -6.988  Mean Steps: 11.0  Avg Duration: 0.032s

Episode 31/500 Summary:
mujoco:  Mean Reward: 348.687  Mean Steps: 337.0  Avg Duration: 0.177s
brax:  Mean Reward: 56.290  Mean Steps: 61.0  Avg Duration: 0.037s

Episode 41/500 Summary:
mujoco:  Mean Reward: 741.189  Mean Steps: 1000.0  Avg Duration: 0.201s
